In [1]:
import numpy as np
import scipy.stats
import pandas as pd

In [2]:
def getExcelFrame(**kwargs):
    return pd.read_clipboard(**dict(kwargs, header = kwargs.get("header", None))).as_matrix()

In [3]:
def pasteExcelFrame(x, header = False, index = False):
    pd.DataFrame(x).to_clipboard(header, index)

In [ ]:
time_steps = 10*20
date0 = 42680
dateT = 49985
t = np.linspace(date0, dateT, time_steps + 1)
#t = np.arange(date0, dateT, 0.25)
#time_steps = len(t) - 1

time_increment = np.ediff1d(t)/365.25
time_increment_sqrt = np.sqrt(time_increment)

# kappa
kappa = 10.0/100.0

# Local Vol 
a = 1.008013828832
b = 0.021415362451
c = 0.373827531444

sim_num = 1000
simulations = np.zeros((sim_num, 2))
extrange_brownians = np.empty((0,))
extrange_path = np.empty(0)
for path in range(sim_num):
    x = 0
    y = 0
    brownians = np.random.normal(0, scale = time_increment_sqrt)
    current_path = np.empty((0,2))
    for delta, w in zip(time_increment, brownians):
        local_vol = a*(b + c*x)
        x += (y - kappa*x)*delta + local_vol*w
        y += (local_vol**2 - 2*kappa*y)*delta
        current_path = np.vstack((current_path, [x,y]))
        #print x, y, w
        
    if not np.isfinite(x):
        extrange_brownians = brownians
        extrange_path = current_path
        
    simulations[path, :] = [x, y]

In [ ]:
isNan = np.isnan(simulations).sum(axis = 1) == 1
isInf = np.isinf(simulations).sum(axis = 1) == 1
simulations = simulations[np.isfinite(simulations).prod(axis = 1) == 1]
simulations = simulations[(simulations < 100).prod(axis = 1)]

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

plt.close('all')
f, axarr = plt.subplots(2)

num_bins = 50
n, bins, patches = axarr[0].hist(simulations[:, 0], num_bins, normed=1, facecolor='green', alpha=0.5)
axarr[0].set_title('X Histogram')
n, bins, patches = axarr[1].hist(simulations[:, 1], num_bins, normed=1, facecolor='green', alpha=0.5)
axarr[1].set_title('Y Histogram')
plt.show()

In [ ]:
greater_moments = np.array([scipy.stats.moment(simulations, moment=x, axis=0) for x in (2,3)])
means = np.mean(simulations, 0)
moments = np.vstack((means, greater_moments))
moments[1,1] = np.cov(simulations[:,0], simulations[:,1])[0,1]
print "first moment: ", means
print "second moment: ", greater_moments[0,:]
print "third moment: ", greater_moments[1,:]

In [ ]:
experimental_values = getExcelFrame().reshape(2,3).T

In [ ]:
experimental_values

In [ ]:
significan_level = 1.0000000827403710e-011
norm_value = scipy.stats.norm.ppf([significan_level/2.0, 1.0 - significan_level/2.0])
simulations_quantiles = np.percentile(simulations, [100*significan_level/2.0, 100*(1.0 - significan_level/2.0)])

In [ ]:
x = extrange_path[:,0]
y = extrange_path[:,1]
pasteExcelFrame(np.vstack((x, y, extrange_brownians, a*(b+c*x), (y - kappa*x)*0.50)).T)

In [23]:
time_steps = 1000*20
date0 = 42680
dateT = 49985
dates = np.linspace(date0, dateT, time_steps + 1)
time_basis = 325.25
time_increment = np.ediff1d(dates)/time_basis
np.delete(dates, 0)

# kappa
kappa = 3.0/100.0

# Local Vol 
a = 0.390061086639
b = 0.021415362451
c = 0.456175152234

sim_num = 1000000
simulations = np.zeros((sim_num, 2))
extrange_brownians = np.empty((0,))
extrange_path = np.empty(0)
for path in range(sim_num):
    x = 0
    y = 0
    brownians = np.random.normal(0, scale = np.sqrt(time_increment))
    current_path = np.empty((0,2))
    for date, delta, w in zip(dates, time_increment, brownians):
        G = (1.0 - np.exp(-kappa*(dateT - date)/time_basis))/kappa
        local_vol = a*(b + c*x)
        x += (y - kappa*x - local_vol**2*G)*delta + local_vol*w
        y += (local_vol**2 - 2*kappa*y)*delta
        current_path = np.vstack((current_path, [x,y]))
        #print x, y, w
        
    if not np.isfinite(x):
        extrange_brownians = brownians
        extrange_path = current_path
        
    simulations[path, :] = [x, y]

KeyboardInterrupt: 

In [6]:
simulations.max(axis = 0)

array([ 0.95244795,  0.1825793 ])

In [11]:
extrange_path

array([], dtype=float64)

In [ ]:
pd.DataFrame(simulations).to_csv(r"C:\Users\e022434\cheyette.csv" , index = False, header = False, mode = 'w+')

In [ ]:
a